<a href="https://colab.research.google.com/github/Utkarsha1a/Innovate48/blob/main/TrainingData2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 82.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184491 sha256=87a7fba60ad9090116a07bf1bd8447c253bfa20bb0e7959e37c17ababe813ce3
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


In [ ]:
from google.colab import files
uploaded = files.upload()  #Upload .htm files name will be gov_Archives_edgar_data_312070_000148105723004452_form424b2.htm

Saving 452_form424b2.htm to 452_form424b2.htm
Saving 462_form424b2.htm to 462_form424b2.htm
Saving 467_form424b2.htm to 467_form424b2.htm
Saving 477_form424b2.htm to 477_form424b2.htm
Saving 504_dp195976_fwp-5052wfts.htm to 504_dp195976_fwp-5052wfts.htm


In [ ]:
# List down all htm files in colab directory
import os
data = []
for filename in os.listdir('/content/'):
    if filename.endswith("htm"):
        print(filename)
        data.append(filename)


000148105723004452_form424b2.htm
000148105723004477_form424b2.htm
000148105723004462_form424b2.htm
000095010323009504_dp195976_fwp-5052wfts.htm
000148105723004467_form424b2.htm


In [ ]:
 # Create a new Word document from htm file

from docx import Document
from bs4 import BeautifulSoup
import chardet

def html_to_docx(html_file, output_file):
    # Read the HTML file and detect the encoding
    with open(html_file, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    # Create a new Word document
    doc = Document()
    # Convert the HTML to plain text using the detected encoding
    html_content = raw_data.decode(encoding)
    soup = BeautifulSoup(html_content, 'html.parser')
    plain_text = soup.get_text()
    # Add the plain text content to the Word document
    doc.add_paragraph(plain_text)
    # Save the Word document
    doc.save(output_file)


# Convert HTML to Word
for html_file in data:
    print(html_file)
    output_file=html_file.replace("htm", "docx")
    html_to_docx(html_file, output_file)
    print(output_file)



000148105723004452_form424b2.htm
000148105723004452_form424b2.docx
000148105723004477_form424b2.htm
000148105723004477_form424b2.docx
000148105723004462_form424b2.htm
000148105723004462_form424b2.docx
000095010323009504_dp195976_fwp-5052wfts.htm
000095010323009504_dp195976_fwp-5052wfts.docx
000148105723004467_form424b2.htm
000148105723004467_form424b2.docx


In [ ]:
from docx import Document
output = []
for filename in os.listdir('/content/'):
    if filename.endswith("docx"):
        output.append(filename)

def extract_text_from_docx(docx_file):
    # Create a Document object from the Word file
    doc = Document(docx_file)
    # Extract text from paragraphs
    text = []
    for paragraph in doc.paragraphs:
        text.append(paragraph.text)
    # Join the extracted text into a single string
    extracted_text = '\n'.join(text)
    return extracted_text

# Usage example
#docx_file = 'output.docx'
#extracted_text = extract_text_from_docx(docx_file)
#print(extracted_text)

content={}
for file in output:
    modified_filename = file.replace('.docx', '')
    extracted_text = extract_text_from_docx(file)
    # Remove empty lines and convert to a single line with space-separated words
    extracted_text = ' '.join(line.strip() for line in extracted_text.splitlines() if line.strip())
    #content.append(extracted_text)
    content[modified_filename] = extracted_text

#content dictonary :- key = filename & value = extracted data

for key in content.keys():
    print(key)

000148105723004462_form424b2
000148105723004467_form424b2
000095010323009504_dp195976_fwp-5052wfts
000148105723004452_form424b2
000148105723004477_form424b2


In [ ]:
for val in content.values():
    print(val)

In [ ]:
#Upload Barclays.csv here

from google.colab import files
uploaded = files.upload()

Saving Barclays.csv to Barclays (2).csv


In [ ]:
import pandas as pd
import json

In [ ]:
csv_file = 'Barclays.csv'  # Replace 'data.csv' with the path to your CSV file

df = pd.read_csv(csv_file, encoding='ISO-8859-1')

df['NewColumn'] = df['FilerPageLink'].str.extract(r'\/(\d+\/[\w.-]+)$')[0].str.replace('/', '_').str.replace('.htm$', '')
df['ExtractedText'] = df['NewColumn'].apply(lambda x: content[x] if x in content else '')


<ipython-input-87-48a7c84166e5>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['NewColumn'] = df['FilerPageLink'].str.extract(r'\/(\d+\/[\w.-]+)$')[0].str.replace('/', '_').str.replace('.htm$', '')


In [ ]:
print(df['FileName'])

0    W:\archive\2023\02384722.htm
1    W:\archive\2023\02384724.htm
2    W:\archive\2023\02384714.htm
3    W:\archive\2023\02384718.htm
4    W:\archive\2023\02384717.htm
Name: FileName, dtype: object


In [ ]:
csv_file = 'Barclays.csv'  # Replace 'data.csv' with the path to your CSV file
json_file = 'traindata.json'  # Replace 'data.json' with the desired output JSON file

data_list = []

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    data = {
        "context": row["ExtractedText"],
        "qas": [
            {
                "question": "What is the CUSIPno?",
                "answers": [{"text":row["CUSIP"]}]
            },
            {
                "question": "What is the Maturity date?",
                "answers": [{"text":row["MaturityDate"]}]
            },
            {
                "question": "What is the Maturity date?",
                "answers": [{"text":row["IssueDate"]}]
            },
        ]
    }
    data_list.append(data)

# Print the data structure for each row
for data in data_list:
    print(data)

with open(json_file, 'w') as file:
  json.dump(data_list, file, indent=4)

{'context': '000148105723004452_form424b2', 'qas': [{'question': 'What is the CUSIPno?', 'answer': '06745MMC9 / US06745MMC90'}, {'question': 'What is the Maturity date?', 'answer': 'July 3, 2025'}, {'question': 'What is the Maturity date?', 'answer': 'July 6, 2023'}]}
{'context': '000095010323009504_dp195976_fwp-5052wfts', 'qas': [{'question': 'What is the CUSIPno?', 'answer': nan}, {'question': 'What is the Maturity date?', 'answer': nan}, {'question': 'What is the Maturity date?', 'answer': nan}]}
{'context': '000148105723004462_form424b2', 'qas': [{'question': 'What is the CUSIPno?', 'answer': '06745MMH8 / US06745MMH87'}, {'question': 'What is the Maturity date?', 'answer': 'July 2, 2026'}, {'question': 'What is the Maturity date?', 'answer': 'July 3, 2023'}]}
{'context': '000148105723004477_form424b2', 'qas': [{'question': 'What is the CUSIPno?', 'answer': '06741W6C9 / US06741W6C99'}, {'question': 'What is the Maturity date?', 'answer': 'July 3, 2025'}, {'question': 'What is the Ma